# Laboratório Prático: Vision Transformers

Este notebook demonstra a implementação prática dos Vision Transformers, desde sua construção básica até aplicações avançadas. Vamos explorar a arquitetura, funcionamento e aplicações desses modelos revolucionários de visão computacional.

## 1. Setup e Instalação de Dependências

Primeiro, vamos garantir que temos todas as bibliotecas necessárias instaladas:

In [ ]:
# Descomente para instalar as dependências
!pip install torch torchvision timm transformers matplotlib einops opencv-python tqdm pillow scikit-learn

In [1]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange
from tqdm.auto import tqdm

# Verifique se temos GPU disponível
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

ModuleNotFoundError: No module named 'einops'

## 2. Implementação do Vision Transformer (ViT) do Zero

Vamos implementar um Vision Transformer passo a passo para entender cada componente.

### 2.1 Componentes Básicos do Transformer

In [ ]:
class PatchEmbedding(nn.Module):
    """Converte uma imagem em embeddings de patches."""
    
    def __init__(self, in_channels=3, patch_size=16, emb_dim=768, img_size=224):
        super().__init__()
        self.patch_size = patch_size
        # Número de patches em cada dimensão
        self.num_patches = (img_size // patch_size) ** 2
        # Projeção linear (implementada como convolução)
        self.proj = nn.Conv2d(
            in_channels, 
            emb_dim, 
            kernel_size=patch_size, 
            stride=patch_size
        )
        
    def forward(self, x):
        # x: [batch, channels, height, width]
        x = self.proj(x)  # [batch, emb_dim, num_patches^0.5, num_patches^0.5]
        x = rearrange(x, 'b c h w -> b (h w) c')  # [batch, num_patches, emb_dim]
        return x


class AttentionBlock(nn.Module):
    """Implementação do bloco Multi-Head Self-Attention"""
    
    def __init__(self, emb_dim=768, num_heads=12, dropout=0.0):
        super().__init__()
        self.ln = nn.LayerNorm(emb_dim)
        self.mha = nn.MultiheadAttention(
            embed_dim=emb_dim,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x_ln = self.ln(x)
        attn_output, _ = self.mha(x_ln, x_ln, x_ln)
        return x + self.dropout(attn_output)


class MLPBlock(nn.Module):
    """Bloco MLP com GELU"""
    
    def __init__(self, emb_dim=768, mlp_dim=3072, dropout=0.0):
        super().__init__()
        self.ln = nn.LayerNorm(emb_dim)
        self.mlp = nn.Sequential(
            nn.Linear(emb_dim, mlp_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_dim, emb_dim),
            nn.Dropout(dropout)
        )
        
    def forward(self, x):
        return x + self.mlp(self.ln(x))


class TransformerBlock(nn.Module):
    """Bloco Transformer: Attention + MLP"""
    
    def __init__(self, emb_dim=768, num_heads=12, mlp_dim=3072, dropout=0.0):
        super().__init__()
        self.attention = AttentionBlock(emb_dim, num_heads, dropout)
        self.mlp = MLPBlock(emb_dim, mlp_dim, dropout)
        
    def forward(self, x):
        x = self.attention(x)
        x = self.mlp(x)
        return x

### 2.2 Arquitetura Completa do Vision Transformer

In [ ]:
class VisionTransformer(nn.Module):
    """Implementação do Vision Transformer (ViT)"""
    
    def __init__(
        self, 
        img_size=224, 
        patch_size=16, 
        in_channels=3, 
        num_classes=1000,
        emb_dim=768, 
        depth=12, 
        num_heads=12, 
        mlp_dim=3072, 
        dropout=0.1
    ):
        super().__init__()
        
        # Embedding de patches
        self.patch_embed = PatchEmbedding(
            in_channels=in_channels,
            patch_size=patch_size,
            emb_dim=emb_dim,
            img_size=img_size
        )
        num_patches = self.patch_embed.num_patches
        
        # Token de classe (CLS) e embedding posicional
        self.cls_token = nn.Parameter(torch.zeros(1, 1, emb_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, emb_dim))
        self.dropout = nn.Dropout(dropout)
        
        # Blocos Transformer
        self.blocks = nn.Sequential(
            *[TransformerBlock(emb_dim, num_heads, mlp_dim, dropout) for _ in range(depth)]
        )
        
        # Layer norm final e classificador
        self.ln = nn.LayerNorm(emb_dim)
        self.head = nn.Linear(emb_dim, num_classes)
        
        # Inicialização dos parâmetros
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        nn.init.trunc_normal_(self.cls_token, std=0.02)
        self.apply(self._init_weights)
    
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=0.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
    
    def forward(self, x):
        # x: [batch_size, channels, height, width]
        batch_size = x.shape[0]
        
        # Embedding de patches
        x = self.patch_embed(x)  # [batch_size, num_patches, emb_dim]
        
        # Adiciona token de classe
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)  # [batch_size, 1, emb_dim]
        x = torch.cat((cls_tokens, x), dim=1)  # [batch_size, num_patches+1, emb_dim]
        
        # Adiciona embedding posicional
        x = x + self.pos_embed  # [batch_size, num_patches+1, emb_dim]
        x = self.dropout(x)
        
        # Passa pelos blocos Transformer
        x = self.blocks(x)  # [batch_size, num_patches+1, emb_dim]
        x = self.ln(x)  # [batch_size, num_patches+1, emb_dim]
        
        # Usa o token CLS para classificação
        x = x[:, 0]  # [batch_size, emb_dim]
        x = self.head(x)  # [batch_size, num_classes]
        
        return x

### 2.3 Visualização do Processo de Patching

In [ ]:
def load_and_preprocess_image(img_path, img_size=224):
    """Carrega e pré-processa uma imagem para visualização."""
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor()
    ])
    img = Image.open(img_path).convert('RGB')
    return img, transform(img)

def visualize_patches(img_tensor, patch_size=16):
    """Visualiza o processo de divisão de imagem em patches."""
    # Convertendo o tensor para numpy para visualização
    img_np = img_tensor.permute(1, 2, 0).numpy()
    
    # Obtém dimensões da imagem
    h, w, c = img_np.shape
    
    # Criando grade de patches
    fig, axs = plt.subplots(h // patch_size, w // patch_size, figsize=(10, 10))
    
    for i in range(0, h, patch_size):
        for j in range(0, w, patch_size):
            # Extrai um patch
            patch = img_np[i:i+patch_size, j:j+patch_size, :]
            # Mostra o patch na posição correta da grade
            axs[i//patch_size, j//patch_size].imshow(patch)
            axs[i//patch_size, j//patch_size].axis('off')
    
    plt.tight_layout()
    plt.suptitle(f'Imagem dividida em patches de {patch_size}x{patch_size}', fontsize=16)
    plt.subplots_adjust(top=0.94)
    plt.show()

# Para demonstrar, baixe uma imagem de exemplo
import urllib.request
from pathlib import Path

# Cria diretório de dados se não existir
data_dir = Path('data')
data_dir.mkdir(exist_ok=True)

# URL da imagem de exemplo (Creative Commons)
img_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/49/Golden_retriever_standing_in_a_park.jpg/640px-Golden_retriever_standing_in_a_park.jpg"
img_path = data_dir / "sample_dog.jpg"

# Baixa a imagem se ela não existir localmente
if not img_path.exists():
    urllib.request.urlretrieve(img_url, img_path)
    print(f"Imagem baixada para {img_path}")
else:
    print(f"Usando imagem existente em {img_path}")

# Visualiza a imagem original e seus patches
img, img_tensor = load_and_preprocess_image(img_path)
plt.figure(figsize=(6, 6))
plt.imshow(img)
plt.title("Imagem Original")
plt.axis('off')
plt.show()

# Visualiza os patches
visualize_patches(img_tensor, patch_size=16)

### 2.4 Inicialização e Inferência com ViT

In [ ]:
# Criando um ViT com configurações reduzidas para fins de demonstração
tiny_vit = VisionTransformer(
    img_size=224,
    patch_size=16,
    in_channels=3,
    num_classes=1000,
    emb_dim=192,        # Reduzido de 768
    depth=4,            # Reduzido de 12
    num_heads=3,        # Reduzido de 12
    mlp_dim=768,        # Reduzido de 3072
    dropout=0.1
)

# Movendo para o dispositivo disponível
tiny_vit = tiny_vit.to(device)

# Contando parâmetros
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Parâmetros treináveis do modelo: {count_parameters(tiny_vit):,}")

# Gerando uma entrada aleatória para testar o modelo
batch_size = 4
x = torch.randn(batch_size, 3, 224, 224).to(device)

# Executando inferência
with torch.no_grad():
    output = tiny_vit(x)

print(f"Shape da entrada: {x.shape}")
print(f"Shape da saída: {output.shape}")

## 3. Usando Modelos Pré-Treinados com HuggingFace e timm

### 3.1 Carregando um ViT Pré-treinado

In [ ]:
import timm
from transformers import ViTForImageClassification, ViTImageProcessor

# Carregando modelo usando timm
print("Modelos ViT disponíveis em timm:")
vit_models = [m for m in timm.list_models() if 'vit' in m]
for i, m in enumerate(vit_models[:10]):  # Mostrar apenas os primeiros 10
    print(f"  {i+1}. {m}")
print(f"... e mais {len(vit_models)-10} modelos")

# Carregando um modelo ViT-Base pré-treinado no ImageNet
model_timm = timm.create_model('vit_base_patch16_224', pretrained=True)
model_timm.eval().to(device)

# Carregando o mesmo modelo usando HuggingFace Transformers
model_name = 'google/vit-base-patch16-224'
processor = ViTImageProcessor.from_pretrained(model_name)
model_hf = ViTForImageClassification.from_pretrained(model_name).to(device)

### 3.2 Preparando Imagens para Inferência

In [ ]:
def prepare_image(img_path, processor=None):
    """Prepara uma imagem para inferência em ViT."""
    if processor:
        # Usando o processador do HuggingFace
        image = Image.open(img_path).convert('RGB')
        inputs = processor(images=image, return_tensors="pt")
        return inputs.pixel_values.to(device)
    else:
        # Preparação padrão para timm
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        img = Image.open(img_path).convert('RGB')
        return transform(img).unsqueeze(0).to(device)

# Baixa ImageNet labels para interpretação da saída
import urllib.request
import json

# Tentar baixar labels do ImageNet se não existirem localmente
labels_path = 'data/imagenet_labels.json'
if not os.path.exists(labels_path):
    os.makedirs(os.path.dirname(labels_path), exist_ok=True)
    try:
        url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
        urllib.request.urlretrieve(url, labels_path)
        print(f"Labels baixados para {labels_path}")
    except Exception as e:
        print(f"Erro ao baixar labels: {e}")
        # Criar um arquivo vazio de fallback
        with open(labels_path, 'w') as f:
            json.dump([f"class_{i}" for i in range(1000)], f)

# Carregando os labels
with open(labels_path, 'r') as f:
    labels = json.load(f)

### 3.3 Realizando Inferência com Modelos Pré-treinados

In [ ]:
def predict_with_vit(img_path, model, processor=None, top_k=5):
    """Faz previsões em uma imagem usando modelo ViT."""
    # Prepara a imagem
    img_tensor = prepare_image(img_path, processor)
    
    # Exibe a imagem original
    img = Image.open(img_path).convert('RGB')
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    
    # Inferência
    with torch.no_grad():
        if hasattr(model, 'forward') and 'pixel_values' in model.forward.__code__.co_varnames:
            # Modelo HuggingFace
            outputs = model(pixel_values=img_tensor)
            logits = outputs.logits
        else:
            # Modelo timm
            logits = model(img_tensor)
    
    # Obtém as probabilidades com softmax
    probs = F.softmax(logits, dim=1)[0]
    
    # Pega as top-k classes
    top_probs, top_idxs = torch.topk(probs, top_k)
    
    # Mostra os resultados
    for i, (idx, prob) in enumerate(zip(top_idxs.cpu().numpy(), top_probs.cpu().numpy())):
        print(f"{i+1}. {labels[idx]}: {prob:.4f} ({100*prob:.2f}%)")

# Usa a imagem de cachorro baixada anteriormente
print("\nPrevisões com modelo timm:")
predict_with_vit(img_path, model_timm)

print("\nPrevisões com modelo HuggingFace:")
predict_with_vit(img_path, model_hf, processor)

## 4. Visualizando a Atenção em Vision Transformers

Uma das vantagens dos transformers é a capacidade de interpretar como o modelo "olha" para a imagem através dos mapas de atenção.

In [ ]:
import cv2
from transformers import ViTForImageClassification
import torch.nn.functional as F

# Carregar modelo com outputs de atenção
model_name = 'google/vit-base-patch16-224'
model_attn = ViTForImageClassification.from_pretrained(model_name, output_attentions=True).to(device)
model_attn.eval()

def get_attention_maps(img_path, model, processor):
    """Extrai mapas de atenção de um modelo ViT para uma imagem."""
    # Prepara a imagem
    img_tensor = prepare_image(img_path, processor)
    
    # Inferência com saídas de atenção
    with torch.no_grad():
        outputs = model(pixel_values=img_tensor, output_attentions=True)
    
    # Extrai atenções - shape é (batch, num_heads, seq_len, seq_len)
    attentions = outputs.attentions
    
    return attentions

def visualize_attention(img_path, attentions, layer=11, head=0):
    """Visualiza um mapa de atenção específico sobre uma imagem."""
    # Carrega a imagem
    img = Image.open(img_path).convert('RGB')
    img = img.resize((224, 224))
    img_np = np.array(img)
    
    # Extrai atenção do token CLS para patches
    # O tensor de atenção tem shape (batch, num_heads, seq_len, seq_len)
    # O CLS token está na posição 0, então pegamos a primeira linha
    attention = attentions[layer][0, head, 0, 1:].reshape(14, 14)
    
    # Normaliza valores de atenção para [0, 1]
    attention_resized = F.interpolate(attention.unsqueeze(0).unsqueeze(0), 
                             size=(224, 224), mode='bilinear', align_corners=False)
    attention_resized = attention_resized.squeeze().cpu().numpy()
    
    # Visualiza
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    
    ax1.imshow(img_np)
    ax1.set_title("Imagem Original")
    ax1.axis('off')
    
    ax2.imshow(attention_resized, cmap='viridis')
    ax2.set_title(f"Mapa de Atenção (Layer {layer}, Head {head})")
    ax2.axis('off')
    
    # Sobreposição
    attention_heatmap = np.uint8(plt.cm.viridis(attention_resized)[:, :, :3] * 255)
    overlay = cv2.addWeighted(img_np, 0.6, attention_heatmap, 0.4, 0)
    ax3.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    ax3.set_title("Sobreposição")
    ax3.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return attention

# Extrai mapas de atenção para a imagem de cachorro
attentions = get_attention_maps(img_path, model_attn, processor)

# Visualiza a atenção em diferentes camadas e cabeças
for layer in [0, 5, 11]:  # Início, meio e fim do modelo
    for head in [0, 5]:  # Diferentes cabeças de atenção
        _ = visualize_attention(img_path, attentions, layer=layer, head=head)
        print(f"Camada {layer}, Cabeça {head}: Observe como diferentes cabeças atendem a diferentes aspectos da imagem")

## 5. Fine-tuning de ViT em um Dataset Personalizado

Vamos demonstrar como fazer fine-tuning de um Vision Transformer pré-treinado para uma tarefa específica usando o dataset CIFAR-10.

In [ ]:
# Função para carregar o CIFAR-10
def load_cifar10(batch_size=64):
    # Transformações para o CIFAR-10 (redimensionar para 224x224 para o ViT)
    transform_train = transforms.Compose([
        transforms.Resize(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    
    transform_test = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    
    # Carrega datasets
    train_dataset = torchvision.datasets.CIFAR10(
        root='./data', train=True, download=True, transform=transform_train
    )
    
    test_dataset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=transform_test
    )
    
    # Nomes das classes
    class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                   'dog', 'frog', 'horse', 'ship', 'truck']
    
    # Criar dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    return train_loader, test_loader, class_names

# Vamos ver algumas imagens do dataset
train_loader, _, class_names = load_cifar10(batch_size=4)
examples = iter(train_loader)
images, labels = next(examples)

# Mostra algumas imagens
plt.figure(figsize=(12, 6))
for i in range(4):
    plt.subplot(1, 4, i+1)
    # Desfaz a normalização para visualização
    img = images[i].permute(1, 2, 0).numpy()
    img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    plt.title(class_names[labels[i]])
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
def fine_tune_vit(num_epochs=3, batch_size=32, lr=1e-4):
    """Fine-tune um ViT no dataset CIFAR-10."""
    # Carrega dados
    train_loader, test_loader, class_names = load_cifar10(batch_size)
    num_classes = len(class_names)
    
    # Modelo: vamos usar um ViT pequeno pré-treinado
    print("Carregando modelo pré-treinado...")
    model = timm.create_model('vit_small_patch16_224', pretrained=True)
    
    # Modifica a última camada para o número correto de classes
    model.head = nn.Linear(model.head.in_features, num_classes)
    model = model.to(device)
    
    # Optimizer: AdamW com weight decay
    optimizer = torch.optim.AdamW([
        # Parâmetros da cabeça com LR mais alto
        {'params': model.head.parameters(), 'lr': lr * 10},
        # Demais parâmetros com LR padrão
        {'params': [p for n, p in model.named_parameters() if 'head' not in n]}
    ], lr=lr, weight_decay=0.01)
    
    # Scheduler de taxa de aprendizagem
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    # Função de perda
    criterion = nn.CrossEntropyLoss()
    
    # Métricas de treinamento
    best_acc = 0.0
    train_losses, train_accs = [], []
    val_losses, val_accs = [], []
    
    # Loop de treinamento
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        
        # Loop de treinamento
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass e otimização
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Estatísticas
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            train_bar.set_postfix({'loss': running_loss/(train_bar.n+1), 'acc': 100.*correct/total})
        
        train_loss = running_loss / len(train_loader)
        train_acc = 100. * correct / total
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        
        # Avaliação
        model.eval()
        running_loss, correct, total = 0.0, 0, 0
        
        with torch.no_grad():
            val_bar = tqdm(test_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Val]')
            for images, labels in val_bar:
                images, labels = images.to(device), labels.to(device)
                
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                # Estatísticas
                running_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
                
                val_bar.set_postfix({'loss': running_loss/(val_bar.n+1), 'acc': 100.*correct/total})
        
        val_loss = running_loss / len(test_loader)
        val_acc = 100. * correct / total
        val_losses.append(val_loss)
        val_accs.append(val_acc)
        
        # Atualiza scheduler
        scheduler.step()
        
        # Salva melhor modelo
        if val_acc > best_acc:
            best_acc = val_acc
            # Descomente para salvar o modelo
            # torch.save(model.state_dict(), 'vit_cifar10_best.pth')
        
        print(f'Epoch {epoch+1}/{num_epochs}: '\
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, '\
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
    
    # Plotar resultados do treinamento
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, num_epochs+1), train_losses, 'b-', label='Treino')
    plt.plot(range(1, num_epochs+1), val_losses, 'r-', label='Validação')
    plt.xlabel('Epoch')
    plt.ylabel('Perda')
    plt.title('Evolução da Perda')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(range(1, num_epochs+1), train_accs, 'b-', label='Treino')
    plt.plot(range(1, num_epochs+1), val_accs, 'r-', label='Validação')
    plt.xlabel('Epoch')
    plt.ylabel('Acurácia (%)')
    plt.title('Evolução da Acurácia')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    return model, best_acc

# Descomente para executar o fine-tuning
# Para economizar tempo, use apenas 1 época
# model_finetuned, best_acc = fine_tune_vit(num_epochs=1)

## 6. Utilizando Vision Transformers em Tarefas de Detecção de Objetos

Vamos utilizar o DETR (*DEtection TRansformer*), um modelo que combina Transformers com CNNs para detecção de objetos.

In [ ]:
# Importa DETR do HuggingFace
from transformers import DetrForObjectDetection, DetrImageProcessor
import torch
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

def detect_objects_with_detr(image_path, threshold=0.9):
    """Detecta objetos em uma imagem usando o modelo DETR."""
    # Carrega o modelo e o processador
    try:
        processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
        model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")
        model.to(device)
        
        # Carrega a imagem
        image = Image.open(image_path).convert('RGB')
        
        # Prepara a imagem para o modelo
        inputs = processor(images=image, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Inferência
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Converte saídas para o formato do COCO
        target_sizes = torch.tensor([image.size[::-1]])
        results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=threshold)[0]
        
        # Desenha os resultados na imagem
        draw = ImageDraw.Draw(image)
        
        # COCO classes
        CLASSES = [
            'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
            'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
            'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
            'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
            'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
            'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
            'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
            'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
            'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
            'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
            'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
            'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
            'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
        ]
        
        detections = []
        
        for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
            box = [round(i) for i in box.tolist()]
            class_name = CLASSES[label]
            
            # Cor baseada na classe para melhor visualização
            color = tuple(np.random.randint(0, 255, 3).tolist())
            
            # Desenha caixa
            draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline=color, width=3)
            
            # Texto com classe e score
            text = f"{class_name}: {score.item():.2f}"
            draw.text((box[0], box[1]), text, fill=color)
            
            detections.append({
                "class": class_name,
                "score": score.item(),
                "box": box
            })
        
        # Mostra a imagem com as detecções
        plt.figure(figsize=(15, 12))
        plt.imshow(image)
        plt.axis('off')
        plt.title('Detecção de Objetos com DETR (Transformer-based)')
        plt.show()
        
        # Lista as detecções
        print(f"Detectados {len(detections)} objetos:")
        for i, det in enumerate(detections):
            print(f"{i+1}. {det['class']} (confiança: {det['score']:.2f})")
        
        return image, detections
    
    except Exception as e:
        print(f"Erro ao carregar ou executar o modelo DETR: {e}")
        print("Para usar esta funcionalidade, instale as dependências necessárias:")
        print("pip install transformers timm torch")
        return None, []

# Vamos baixar uma imagem com múltiplos objetos
street_img_url = "https://upload.wikimedia.org/wikipedia/commons/b/bd/Broadway_and_Times_Square_by_night.jpg"
street_img_path = data_dir / "street_scene.jpg"

if not street_img_path.exists():
    urllib.request.urlretrieve(street_img_url, street_img_path)
    print(f"Imagem baixada para {street_img_path}")
else:
    print(f"Usando imagem existente em {street_img_path}")

# Descomente para executar a detecção de objetos
# image_with_detections, detections = detect_objects_with_detr(street_img_path, threshold=0.7)

## 7. Modelos Auto-Supervisionados: Explorando DINO

DINO (Self-Distillation with No Labels) é um método de aprendizado auto-supervisionado para Vision Transformers que produz representações úteis sem necessidade de rótulos.

In [ ]:
def visualize_dino_attention(img_path, threshold=None):
    """Visualiza mapas de atenção do DINO para uma imagem."""
    try:
        # Tenta importar o modelo DINO
        import torch.hub
        
        # Carrega modelo DINO pré-treinado
        print("Carregando modelo DINO...")
        dino_model = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')
        dino_model.eval().to(device)
        
        # Carregar e preprocessar a imagem
        img = Image.open(img_path).convert('RGB')
        img_tensor = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])(img).unsqueeze(0).to(device)
        
        # Extrai atenção do último bloco
        with torch.no_grad():
            # Extrai atenção
            outputs = dino_model.get_last_selfattention(img_tensor)
            
        # Atenção do token CLS para os patches
        attn = outputs[:, :, 0, 1:].reshape(1, -1, 14, 14)
        attn = torch.mean(attn, dim=1).squeeze().cpu().numpy()
        
        # Aplicar thresholding se especificado
        if threshold is not None:
            # Normaliza a atenção para [0, 1]
            attn_norm = (attn - attn.min()) / (attn.max() - attn.min())
            # Aplica threshold
            attn_norm = np.where(attn_norm > threshold, 1.0, 0.0)
            attn = attn_norm
        
        # Redimensiona para o tamanho da imagem
        img = img.resize((224, 224))
        img_np = np.array(img)
        attn_resized = cv2.resize(attn, (img_np.shape[1], img_np.shape[0]))
        
        # Converte a atenção para um mapa de calor
        attn_heatmap = cv2.applyColorMap(np.uint8(255 * attn_resized), cv2.COLORMAP_JET)
        
        # Mistura a imagem original com o mapa de calor
        attn_overlay = cv2.addWeighted(cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR), 0.6, attn_heatmap, 0.4, 0)
        
        # Visualiza
        plt.figure(figsize=(15, 5))
        
        plt.subplot(1, 3, 1)
        plt.imshow(img)
        plt.title("Imagem Original")
        plt.axis('off')
        
        plt.subplot(1, 3, 2)
        plt.imshow(attn_resized, cmap='viridis')
        plt.title("Mapa de Atenção DINO")
        plt.axis('off')
        
        plt.subplot(1, 3, 3)
        plt.imshow(cv2.cvtColor(attn_overlay, cv2.COLOR_BGR2RGB))
        plt.title("Sobreposição")
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
        
        return attn
        
    except Exception as e:
        print(f"Erro ao carregar ou executar o modelo DINO: {e}")
        print("Para usar esta funcionalidade, instale as dependências necessárias:")
        print("pip install torch torchvision matplotlib opencv-python")
        return None

# Descomente para visualizar a atenção DINO em uma imagem
# print("Gerando visualização de atenção do DINO na imagem de cachorro...")
# _ = visualize_dino_attention(img_path)

## 8. Comparação de Performance: ViT vs CNN

Vamos comparar a performance de inferência entre um ViT e uma CNN tradicional como a ResNet.

In [ ]:
import time
import torchvision.models as models

def benchmark_inference(model_name, batch_sizes=[1, 4, 16, 32], input_size=(3, 224, 224), runs=50):
    """Benchmark de inferência para modelos de visão."""
    results = {'batch_size': [], 'latency_ms': [], 'throughput': []}
    
    try:
        # Carregando o modelo
        if model_name == "vit":
            model = timm.create_model('vit_base_patch16_224', pretrained=False)
        elif model_name == "resnet50":
            model = models.resnet50(pretrained=False)
        else:
            raise ValueError(f"Modelo desconhecido: {model_name}")
            
        model.eval().to(device)
        
        for batch_size in batch_sizes:
            # Gera dados de entrada
            dummy_input = torch.randn(batch_size, *input_size).to(device)
            
            # Warm-up
            with torch.no_grad():
                for _ in range(10):
                    _ = model(dummy_input)
            
            # Mede inferência
            latencies = []
            torch.cuda.synchronize() if torch.cuda.is_available() else None
            
            with torch.no_grad():
                for _ in range(runs):
                    start_time = time.time()
                    _ = model(dummy_input)
                    torch.cuda.synchronize() if torch.cuda.is_available() else None
                    latencies.append(time.time() - start_time)
            
            # Calcula métricas
            avg_latency = sum(latencies) / len(latencies) * 1000  # em ms
            imgs_per_sec = batch_size / (avg_latency / 1000)
            
            results['batch_size'].append(batch_size)
            results['latency_ms'].append(avg_latency)
            results['throughput'].append(imgs_per_sec)
            
            print(f"Modelo {model_name}, Batch {batch_size}: {avg_latency:.2f} ms, {imgs_per_sec:.2f} imgs/seg")
        
        return results
    
    except Exception as e:
        print(f"Erro durante benchmark: {e}")
        return results

def plot_benchmark(vit_results, resnet_results):
    """Plota os resultados do benchmark."""
    plt.figure(figsize=(15, 6))
    
    # Latência
    plt.subplot(1, 2, 1)
    plt.plot(vit_results['batch_size'], vit_results['latency_ms'], 'b-o', label='ViT-Base')
    plt.plot(resnet_results['batch_size'], resnet_results['latency_ms'], 'r-o', label='ResNet-50')
    plt.xlabel('Tamanho do Batch')
    plt.ylabel('Latência (ms)')
    plt.title('Comparação de Latência')
    plt.grid(True)
    plt.legend()
    
    # Throughput
    plt.subplot(1, 2, 2)
    plt.plot(vit_results['batch_size'], vit_results['throughput'], 'b-o', label='ViT-Base')
    plt.plot(resnet_results['batch_size'], resnet_results['throughput'], 'r-o', label='ResNet-50')
    plt.xlabel('Tamanho do Batch')
    plt.ylabel('Throughput (imgs/s)')
    plt.title('Comparação de Throughput')
    plt.grid(True)
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Descomente para executar o benchmark
# Se quiser executar benchmarks mais rápidos, reduza o número de runs e batch_sizes
# print("Executando benchmark para ViT...")
# vit_results = benchmark_inference("vit", batch_sizes=[1, 2, 4], runs=10)
# print("\nExecutando benchmark para ResNet-50...")
# resnet_results = benchmark_inference("resnet50", batch_sizes=[1, 2, 4], runs=10)
# plot_benchmark(vit_results, resnet_results)

## 9. Sumário e Conclusões

Neste laboratório, exploramos os Vision Transformers (ViT) em profundidade:

1. **Implementação do Zero**: Construímos um ViT do zero para entender sua arquitetura
2. **Visualização de Patches**: Demonstramos como as imagens são divididas em patches
3. **Modelos Pré-treinados**: Utilizamos ViTs da HuggingFace e timm
4. **Visualização da Atenção**: Exploramos como os ViTs "olham" para as imagens
5. **Fine-tuning**: Adaptamos um ViT para o dataset CIFAR-10
6. **Aplicações Avançadas**: Detectamos objetos com DETR e exploramos o DINO
7. **Performance**: Comparamos ViT e ResNet em termos de latência e throughput

Os Vision Transformers representam uma mudança de paradigma na visão computacional, competindo e frequentemente superando as CNNs tradicionais, especialmente em regimes de dados abundantes e para tarefas que requerem compreensão global de imagens.

No entanto, eles também apresentam desafios em termos de eficiência computacional e escalabilidade para resoluções maiores. Arquiteturas híbridas e otimizadas continuam surgindo para abordar esses desafios, tornando os transformers cada vez mais práticos para aplicações do mundo real.